# Actividad 3.9 – SAGULPA – Previsión de entradas y salidas

El objeto de esta activad es realizar una predicción del número de entradas y salidas en los aparcamientos gestionados por SAGULPA en un día del año. Para lo cual utilizaremos los datos abiertos publicados en su web.
La actividad está dividida en dos fases:

## Fase 1: Generación del dataset.

Título: Datos Abiertos
Url: https://www.sagulpa.com/datos-abiertos

Trabajaremos con el dataset de aparcamientos, utilizando como fuente el fichero del mes de diciembre de 2019. Realizar la ingesta y transformación de datos de forma que podamos obtener las siguientes características en el dataset:

  * Aparcamiento (Nombre del parking)
  * Día
  * Mes
  * Año
  * DíaSemana (L/M/X/J/V/S/D)
  * DíaLectivo (Sí/No)
  * DíaFestivo(Sí/No)
  * Entradas
  * Salidas
  * Hora
  * Temperatura ºCelsius
  * Humedad  %

# Carga del csv.

Cargamos el csv de aparcamientos que está alojado en mi repositorio de github.
Como podemos ver tiene un id de parking, el nombre del parking, un evento de tiempo y uno de entradas y salidas.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
import holidays

df = pd.read_csv('https://raw.githubusercontent.com/alexander6779/SNS/main/Sagulpa/info2019/APARCAMIENTOS-DICIEMBRE-2019.csv',sep=';')
df

,GarageNo,APARCAMIENTO,EventTime,EventDesc
0,8855,ELDER,01/12/2019 0:37,Salida
1,8855,ELDER,01/12/2019 1:17,Salida
2,8855,ELDER,01/12/2019 3:06,Salida
3,8855,ELDER,01/12/2019 8:09,Entrada
4,8855,ELDER,01/12/2019 10:43,Salida
...,...,...,...,...
276159,8601,VEGUETA,31/12/2019 23:10,Entrada
276160,8601,VEGUETA,31/12/2019 23:28,Salida
276161,8601,VEGUETA,31/12/2019 23:28,Entrada
276162,8601,VEGUETA,31/12/2019 23:29,Salida


# Cargar también el csv del clima

Cargamos el csv del clima del mes de diciembre, también está en el repositorio, aclarar que esos datos han sido extraidos de la pagina www.tutiempo.net y han sido comparados con los de la web oficial de la AEMET, pero no hay diferencias entre ellos.Sin embargo, en ambos hay una pérdida de datos del del día 19 a las 21 horas, la cuál la vamos a borrar y para la siguiente fases usaremos un método para generar esos datos.Además para dejar preparado este csv lo que hago es coger la mediana de las horas, es decir a las y media de cada hora,por lo que siempre obtengo temperaturas y humedades de la mediana de las horas.

In [ ]:
temp = pd.read_csv('https://raw.githubusercontent.com/alexander6779/SNS/main/Sagulpa/info2019/tiempo.csv',sep=',')
temp =temp.drop(['web-scraper-order','web-scraper-start-url','Condiciones meteorológicas','Viento','Presión','pagination'], axis=1)
temp = temp.dropna()
temp

,Día,Hora,Temperatura,Humedad
0,Domingo 1 Diciembre 2019,00:00,20°,83%
2,Domingo 1 Diciembre 2019,00:30,20°,83%
3,Domingo 1 Diciembre 2019,01:00,20°,83%
4,Domingo 1 Diciembre 2019,01:30,20°,83%
5,Domingo 1 Diciembre 2019,02:00,20°,78%
...,...,...,...,...
1867,Martes 7 Enero 2020,21:30,18°,73%
1868,Martes 7 Enero 2020,22:00,18°,68%
1869,Martes 7 Enero 2020,22:30,18°,68%
1870,Martes 7 Enero 2020,23:00,18°,68%


In [ ]:
temp.reset_index(drop=True,inplace=True)
index = []
for i in range(len(temp)):
  temp.loc[i]['Día'] =  temp.loc[i]['Día'].replace(' ','-')
  fecha = temp.loc[i]['Día'].split('-')
  fecha[2]= 12
  fecha =  datetime.strptime(fecha[1]+'-'+str(fecha[2])+'-'+fecha[3], '%d-%m-%Y')
  temp.loc[i]['Día'] = fecha.day
  temp.loc[i,'Temperatura'] = temp.loc[i,'Temperatura'].replace('\u00B0','')
  temp.loc[i,'Humedad'] = temp.loc[i,'Humedad'].replace('%','')
  m =  datetime.strptime(temp.loc[i,'Hora'], '%H:%M')
  if m.minute != 30 :
    index.append(i)
  else:
    temp.loc[i,'Hora'] = m.hour
temp = temp.drop(index)
temp.reset_index(drop=True,inplace=True)
temp

,Día,Hora,Temperatura,Humedad
0,1,0,20,83
1,1,1,20,83
2,1,2,20,78
3,1,3,19,83
4,1,4,19,78
...,...,...,...,...
905,7,19,19,68
906,7,20,18,73
907,7,21,18,73
908,7,22,18,68


# Limpieza del parking SAN BERNARDO

He decidio borrar todos los registros que tengan que ver con el parking SAN BERNARDO puesto que hay falta de datos de ese parking debido a un cierre temporal que tuvo dicho parking por construcción en las instalaciones,dicha noticia fue publicada en el periódico digital de la provincia.

No obstante esto sería una primera versión y visto ya hay publicados csv de diciembre de 2021 y 2022, en la próxima versión se podrían añadir nuevas muestras para completar datos.Por cierto, no comento 2020 a causa de la pandemia, dicha situación generó un vacío de datos masivos que no tendrían uso.

In [ ]:
df.drop(df[df['APARCAMIENTO'] == 'SAN BERNARDO'].index, inplace=True)
df.reset_index(drop=True,inplace=True)
df

,GarageNo,APARCAMIENTO,EventTime,EventDesc
0,8855,ELDER,01/12/2019 0:37,Salida
1,8855,ELDER,01/12/2019 1:17,Salida
2,8855,ELDER,01/12/2019 3:06,Salida
3,8855,ELDER,01/12/2019 8:09,Entrada
4,8855,ELDER,01/12/2019 10:43,Salida
...,...,...,...,...
268146,8601,VEGUETA,31/12/2019 23:10,Entrada
268147,8601,VEGUETA,31/12/2019 23:28,Salida
268148,8601,VEGUETA,31/12/2019 23:28,Entrada
268149,8601,VEGUETA,31/12/2019 23:29,Salida


# Creación del dataframe final

Comenzamos con la creación del nuevo dataframe, visto que con el primero que partimos solo obtenemos un día,un parking y un evento en este caso (Entrada o Salida), se ha decidido ampliar horizontes y añadir más columnas para la resolución de los problemas, predecir entradas y salidas.
Por tanto, el dataframe final constará de varias columnas: 



*   parking: nombre del parking.
*   day: día de la semana en número.
*   month: mes del año de tipo numérico.
*   year: año de tipo numérico.
*   hour: hora del día.
*   day_of_week: día de la semana en letra ('L','M','X','J','V','S','D').
*   holiday: se define si es un día festivo o no ('Sí' o 'No').
*   school_day: se define si es un día lectivo o no ('Sí' o 'No').
*   check_in: número de entradas. (inicialización posterior)
*   check_out: número de salidas.(inicialización posterior)
*   temperature: es la temperatura de cada hora del día y cojo la mediana, es decir, que reecojo la temperatura de las y media, se recoge del dataframe del tiempo y se inicializará más tarde.
*   humidity: es el porcentaje de humedad de la hora de ese día, también se inicializa más tarde.

Sabiendo sigue el documento paso a paso.

In [ ]:
df_v2 = pd.DataFrame(columns=['parking','day','month','year','hour','day_of_week','holiday','school_day'])
df_v2['parking'] = df['APARCAMIENTO']
df_v2

,parking,day,month,year,hour,day_of_week,holiday,school_day
0,ELDER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ELDER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ELDER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ELDER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ELDER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
268146,VEGUETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268147,VEGUETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268148,VEGUETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268149,VEGUETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Con el método libre devolvemos 1 si se encuentra en la lista de los festivos importados de una librería de python llamada holidays, se encuentra al principio juntos con otras librerías y 0 si no se encuentras, de ésta manera convertiremos después tales valores en Sí o No.

In [ ]:
def libre(d,m,hol):
  for i,k in enumerate(hol):
    if d == k[0].day and m==k[0].month:
      return 1
  return 0

Método para saber si es un día lectivo o no a partir de un diccionario creado posteriormente, con todos los días no lectivos del mes de diciembre de 2019.

In [ ]:
def no_lectivo(d,l):
  for i,k in enumerate(l):
    if d == k[2]:
      return 0
  return 1

Por consiguientes, empezamos a rellenar nuestro dataframe, añadiendo el día,la hora,el mes, el año, si es lectivo o no, si es festivo o no y el día de la semana en números (1='L',2='M', así hasta el 7), luego los convertimos en categóricos para un mejor entendimiento del usuario.
*Tarda 2 minutos y medio, en mi ordenador, debido a la gran ingesta de datos.

In [ ]:
hol_2019 = sorted(holidays.Spain(subdiv='CN', years=2019).items())
#hol = sorted(holidays.Spain(subdiv='CN', years=2022).items()) utilizo esta línea para el csv de 2022
no_lectivos = {(2019,12,1), (2019,12,7), (2019,12,8), (2019,12,14), 
    (2019,12,15), (2019,12,21), (2019,12,22), (2019,12,30), (2019,12,31), 
    (2019,12,28), (2019,12,29),(2019,12,23), (2019,12,24),(2019,12,25), 
    (2019,12,26), (2019,12,6),(2019,12,9),  (2019,12,27),        
}
"""no_lectivos = {(2022,12,3), (2022,12,4), (2022,12,10), (2022,12,11), 
    (2022,12,6), (2022,12,7), (2022,12,8), (2022,12,17), (2022,12,18), 
    (2022,12,24), (2022,12,28),(2022,12,23), (2022,12,24),(2022,12,25), 
    (2022,12,26), (2022,12,30),(2022,12,29),  (2022,12,27), (2022,12,31)        
} estos son los festivos del 2022"""

for i in range(len(df)):
  date = datetime.strptime(df.loc[i]['EventTime'],'%d/%m/%Y %H:%M')
  a,M,d,h = date.year,date.month,date.day,date.hour 
  day = date.weekday()+1
  df_v2.loc[i]['school_day'] = no_lectivo(d,no_lectivos)
  df_v2.loc[i]['day_of_week'] = day
  df_v2.loc[i]['holiday'] = libre(date.day,date.month,hol_2019)
  df_v2.loc[i]['day'] = d
  df_v2.loc[i]['month'] = M
  df_v2.loc[i]['year'] = a
  df_v2.loc[i]['hour'] = h

Ahora vamos a ir añadiendo a cada registro si hubo una entrada o una salida por cada hora de cada día que devolverá True o False y luego convertiremos en 0 y 1 para poder generar un groupby.

In [ ]:
df_v2['check_in'] = df['EventDesc'] == 'Entrada'
df_v2['check_out'] = df['EventDesc'] == 'Salida'
df_v2['check_in'] = df_v2['check_in'].replace([False,True],[0,1])
df_v2['check_out'] = df_v2['check_out'].replace([False,True],[0,1])
df_v2

,parking,day,month,year,hour,day_of_week,holiday,school_day,check_in,check_out
0,ELDER,1,12,2019,0,7,0,0,0,1
1,ELDER,1,12,2019,1,7,0,0,0,1
2,ELDER,1,12,2019,3,7,0,0,0,1
3,ELDER,1,12,2019,8,7,0,0,1,0
4,ELDER,1,12,2019,10,7,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
268146,VEGUETA,31,12,2019,23,2,0,0,1,0
268147,VEGUETA,31,12,2019,23,2,0,0,0,1
268148,VEGUETA,31,12,2019,23,2,0,0,1,0
268149,VEGUETA,31,12,2019,23,2,0,0,0,1


Usamos el método groupby() de la librería pandas para agrupar el csv por parkig,dia,mes,año,hora,dia festivo y dia lectivo y haciendo una suma total para poder así obtener las entradas y salidas de una cierta hora,día y parking.

In [ ]:
df_v2 = df_v2.groupby(['parking','day','month','year','hour','holiday','day_of_week','school_day'],as_index=False)[['check_in','check_out']].sum()

Tras el agrupamiento,añadimos la humedad y la temperatura con las 2 celdas siguientes, usando el dataframe importado al principio del tiempo y obtenido gracias a la herramienta `web scraper`.
En breve resumen recorro el dataframe del tiempo y añado ambos valores tras comprobar si el día y la hora se encuentra en el nuevo dataframe.

In [ ]:
def addTemp(h,d,t,hu):
  df_v2.loc[(df_v2['hour'] == h) & (df_v2['day'] == d), ['temperature', 'humidity']] = [t, hu]

In [ ]:
temp.reset_index(drop=True,inplace=True)
for i in range(len(temp)):
  h = int(temp.loc[i]['Hora'])
  t = int(temp.loc[i]['Temperatura'])
  hu = int(temp.loc[i]['Humedad'])
  d = int(temp.loc[i]['Día'])
  addTemp(h,d,t,hu)

In [ ]:
df_v2

,parking,day,month,year,hour,holiday,day_of_week,school_day,check_in,check_out,temperature,humidity
0,ELDER,1,12,2019,0,0,7,0,16,23,16.0,29.0
1,ELDER,1,12,2019,1,0,7,0,8,25,17.0,39.0
2,ELDER,1,12,2019,2,0,7,0,1,22,18.0,49.0
3,ELDER,1,12,2019,3,0,7,0,1,23,17.0,63.0
4,ELDER,1,12,2019,4,0,7,0,0,10,17.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4045,VEGUETA,31,12,2019,19,0,2,0,14,44,18.0,56.0
4046,VEGUETA,31,12,2019,20,0,2,0,11,20,17.0,45.0
4047,VEGUETA,31,12,2019,21,0,2,0,10,8,17.0,27.0
4048,VEGUETA,31,12,2019,22,0,2,0,1,3,18.0,26.0


Por último, antes de la descarga, cambio los valores del 1-7 por los días de la semana (L,M,X,J,V,S,D),los días libres y los festivos por Sí o No. 

In [ ]:
df_v2['day_of_week'] = df_v2['day_of_week'].replace([1,2,3,4,5,6,7],['L','M','X','J','V','S','D'])
df_v2['holiday'] = df_v2['holiday'].replace([0,1],['No','Sí'])
df_v2['school_day'] = df_v2['school_day'].replace([0,1],['No','Sí'])
df_v2

,parking,day,month,year,hour,holiday,day_of_week,school_day,check_in,check_out,temperature,humidity
0,ELDER,1,12,2019,0,No,D,No,16,23,16.0,29.0
1,ELDER,1,12,2019,1,No,D,No,8,25,17.0,39.0
2,ELDER,1,12,2019,2,No,D,No,1,22,18.0,49.0
3,ELDER,1,12,2019,3,No,D,No,1,23,17.0,63.0
4,ELDER,1,12,2019,4,No,D,No,0,10,17.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4045,VEGUETA,31,12,2019,19,No,M,No,14,44,18.0,56.0
4046,VEGUETA,31,12,2019,20,No,M,No,11,20,17.0,45.0
4047,VEGUETA,31,12,2019,21,No,M,No,10,8,17.0,27.0
4048,VEGUETA,31,12,2019,22,No,M,No,1,3,18.0,26.0


Finalmente, guardamos en el sistema de ficheros de google colab nuestro nuevo dataframe listo y preparado para explotarlo, para la segunda fase.
No obstante, el csv ya está subido a mi repositorio.

In [ ]:
df_v2.to_csv('Sagulpa.csv',index=False)